We are going to access some market option data, using [pandas](http://pandas.pydata.org/pandas-docs/stable/10min.html#).

In [ ]:
import pandas as pd
from pandas_datareader import data as web
import datetime

from pandas_datareader.data import Options
Symbol = 'AAPL'

yahoo_options = Options(Symbol, 'yahoo')
data = yahoo_options.get_all_data()

`data` is a panda with rows labelled by a [multiindex](http://pandas.pydata.org/pandas-docs/stable/advanced.html), and columns indexed by various labels ('Last','Bid',...).

Dates are handled by the [`datetime` module](https://docs.python.org/3/library/datetime.html).

We can access slices of the panda in various ways (uncomment one of the commands in this list to see what it does).

In [ ]:
#data.loc[(110, slice(None), 'puts'),:].iloc[0:5, 0:8]
#data.loc[(slice(None), "2019-01-18"),:].iloc[0:20, 0:8]
#data.loc[(slice(None), "2019-01-18"),:].iloc[0:20, 0:8].values

We can also access the latest spot price quote from Yahoo.

In [ ]:
Stock_Quote = web.get_quote_yahoo(Symbol)

But we are only interested in the last quoted value:

In [ ]:
Stock = Stock_Quote.loc[:,'last'].values
print('The last quoted value is $%.2f'%Stock[0])

For each expiry time, I want to plot call and put option prices as functions of the strike price

In [ ]:
PriceType = 'IV' # use "Mid", "Bid", "Ask", "Last", or "IV"
if PriceType == 'Mid':
    GetPrices = lambda x: (x.loc[:,'Bid'].values+x.loc[:,'Ask'].values)/2.0
else:
    GetPrices = lambda x: x.loc[:,PriceType].values

Minimum_Open_Interest = 100  # we will only include quotes with sufficiently high open interest
Option_Types = ['puts','calls']
Option_Titles = ['Put','Call']
Expiry_Dates = data.index.levels[1]
Strikes = data.index.levels[0]
# Prepare the plot
from pylab import *
style.use('seaborn-white')
import mpld3 # enables zooming
fig = figure(figsize=(12,8))
for i in range(2):
    ax = subplot(1,2,i+1)
    Legend_Labels = []
    for expiry_date in Expiry_Dates[Expiry_Dates.to_datetime()>datetime.datetime.now()]: # only use future expiry times
        mydate = expiry_date.strftime('%Y-%m-%d') # we'll use this for selecting entries in the dataframe
         
        x = []; y = [] # the values to be plotted
        for Strike in Strikes:
            tmp = data.loc[(Strike,mydate,Option_Types[i]),:] # extract a one-line dataframe
            p = GetPrices(tmp) # this function was defined at the head of the cell
            OI = tmp.loc[:,'Open_Int'].values
            Chg = tmp.loc[:,'Chg'].values
            if len(p)>0 and Chg !=0 and OI>=Minimum_Open_Interest:
                x.append(Strike)
                y.append(p)
        if len(x)>0:  # ie., if there is anything to plot
            Legend_Labels.append(mydate);
            plot(x,y,'o-',clip_on=False);
    xlabel('Strike Price',fontsize=16);
    if PriceType=='IV':
        title('%s Implied Volatilities'%Option_Titles[i],fontsize=18);
    else:
        title('%s Option Prices (%s)'%(Option_Titles[i],PriceType),fontsize=18);
    if i==1:
        ax.annotate('%s: $%.2f' % (Symbol,Stock[0]), xy=(0.5, 0.95), xycoords='figure fraction',
             ha="center", va="top",bbox=dict(boxstyle="square", fc="0.8"),fontsize=14,color='darkblue');
    legend(Legend_Labels,loc=2-i,title='');
    ax.grid(True,alpha=0.3);
savefig("Images/"+Symbol+"Options"+PriceType+".eps")
savefig("Images/"+Symbol+"Options"+PriceType+".pdf")
mpld3.display(fig)
